In [1]:
import json
import findspark
findspark.init()

In [2]:
from pyspark import SparkContext
sc = SparkContext('local[8]', 'Assignment_2')

In [3]:
sample_data = sc.textFile('asnlib/publicdata/small1.csv').map(lambda x: x.split(',')).map(lambda x: (x[0], x[1]))
sample_data.first()

('user_id', 'business_id')

In [4]:
sample_data.take(10)

[('user_id', 'business_id'),
 ('1', '100'),
 ('1', '98'),
 ('1', '101'),
 ('1', '102'),
 ('2', '101'),
 ('2', '99'),
 ('2', '97'),
 ('2', '100'),
 ('3', '98')]

In [5]:
sample_data.count()

75

In [6]:
header = sample_data.first()
sample_data_without_header = sample_data.filter(lambda x: x!= header)

In [7]:
sample_data_without_header.first()

('1', '100')

In [8]:
distinct_sets = sample_data_without_header.distinct()

In [9]:
distinct_sets.count()

74

In [10]:
grouped_sets = distinct_sets.groupByKey().map(lambda x: (x[0], sorted(list(x[1]))))

In [11]:
grouped_sets.collect()

[('1', ['100', '101', '102', '98']),
 ('4', ['101', '102', '103', '97', '99']),
 ('8', ['102', '103', '104', '97', '98', '99']),
 ('9', ['97', '98', '99']),
 ('10', ['97', '98']),
 ('12', ['100', '101', '102', '105', '106', '107', '108', '98']),
 ('16', ['100', '101', '98', '99']),
 ('19', ['102', '97', '98']),
 ('14', ['97']),
 ('17', ['97', '99']),
 ('2', ['100', '101', '97', '99']),
 ('3', ['102', '103', '105', '97', '98', '99']),
 ('5', ['97', '98']),
 ('6', ['101', '102']),
 ('7', ['101', '97', '99']),
 ('11', ['97', '98', '99']),
 ('13', ['100', '101', '102', '103', '105', '106', '107', '108', '98', '99']),
 ('15', ['101', '97', '99']),
 ('18', ['97', '98', '99'])]

In [12]:
grouped_sets.glom().map(len).collect()

[10, 9]